In [7]:
import heapq
import numpy as np

# Define the goal state for the 8-puzzle
GOAL_STATE = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

class Puzzle:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.state = np.array(state)
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost + self.manhattan_distance()

    def __lt__(self, other):
        return self.cost < other.cost

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    def manhattan_distance(self):
        """Calculate the Manhattan distance heuristic."""
        distance = 0
        for i in range(1, 9):  # Ignore the empty tile (0)
            x, y = np.where(self.state == i)
            goal_x, goal_y = np.where(GOAL_STATE == i)
            x, y = x[0], y[0]
            goal_x, goal_y = goal_x[0], goal_y[0]
            distance += abs(x - goal_x) + abs(y - goal_y)
        return int(distance)

    def possible_moves(self):
        """Generate possible moves from the current state."""
        x, y = np.where(self.state == 0)
        x, y = x[0], y[0]
        moves = []
        if x > 0:
            moves.append(('Up', (x - 1, y)))
        if x < 2:
            moves.append(('Down', (x + 1, y)))
        if y > 0:
            moves.append(('Left', (x, y - 1)))
        if y < 2:
            moves.append(('Right', (x, y + 1)))
        return moves

    def generate_child(self, move, new_pos):
        """Generate a new puzzle state by moving the blank tile."""
        new_state = self.state.copy()
        x, y = np.where(new_state == 0)
        x, y = x[0], y[0]
        new_x, new_y = new_pos
        new_state[x, y], new_state[new_x, new_y] = new_state[new_x, new_y], new_state[x, y]
        return Puzzle(new_state, parent=self, move=move, depth=self.depth + 1, cost=self.depth + 1)

    def is_goal(self):
        """Check if the current state is the goal state."""
        return np.array_equal(self.state, GOAL_STATE)

    def path(self):
        """Trace back the path from the goal state to the initial state."""
        path = []
        current = self
        while current.parent:
            path.append(current.move)
            current = current.parent
        return path[::-1]

# A* Search Algorithm

def a_star(initial_state):
    """Solve the 8-puzzle using A* search."""
    initial_puzzle = Puzzle(initial_state)
    open_list = []
    closed_set = set()
    heapq.heappush(open_list, initial_puzzle)

    while open_list:
        current = heapq.heappop(open_list)
        if current.is_goal():
            return current.path(), current.state
        closed_set.add(tuple(map(tuple, current.state)))

        for move, new_pos in current.possible_moves():
            child = current.generate_child(move, new_pos)
            if tuple(map(tuple, child.state)) not in closed_set:
                heapq.heappush(open_list, child)

    return None, None  # No solution found

# Google Colab Input Handling
print("Enter the 8-puzzle numbers row-wise (use 0 for empty space, separated by spaces):")
user_input = input().strip()
user_input = list(map(int, user_input.split()))
initial_state = np.array(user_input).reshape(3, 3)

# Solve the puzzle
solution, final_state = a_star(initial_state)
if solution:
    print("Solution found:", solution)
    print("Final arranged puzzle:")
    print(final_state)
else:
    print("No solution exists")


Enter the 8-puzzle numbers row-wise (use 0 for empty space, separated by spaces):
6 5 4 7 8 2 3 1 0
Solution found: ['Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Right', 'Down', 'Left', 'Down', 'Right']
Final arranged puzzle:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
